In [56]:
from typing import Dict, Optional

from langchain_core.callbacks import CallbackManagerForChainRun

from chains.chaining import *
from prompt.prompt_template import *
from llms.gpt import *
from langchain_script import *
import dotenv
dotenv.load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
llm_client = OpenAI(model_name="gpt-3.5-turbo",openai_api_key=OPENAI_API_KEY, temperature=0.7)

In [57]:
content_gen = extract_text_from_pdf_page_by_page(os.path.join(docs_dir, "report.pdf"))
content_chunks = split_large_text(content_gen)
prompt1_txt = document_reader(os.path.join(prompts_dir,'prompt1.txt'))
prompt2_txt = document_reader(os.path.join(prompts_dir,'prompt2.txt'))
prompt3_txt = document_reader(os.path.join(prompts_dir,'prompt3.txt'))
prompt4_txt = document_reader(os.path.join(prompts_dir,'prompt4.txt'))

In [58]:
# writing_style = llm_chaining(llm=llm_client,prompt=initial_prompt_template,output_key="writing")
# research_quality = llm_chaining(llm=llm_client,prompt=chain_prompt_template,output_key="quality")
# flaw_report = llm_chaining(llm=llm_client,prompt=chain_prompt_template,output_key="flaws")
# suggestions = llm_chaining(llm=llm_client,prompt=chain_prompt_template,output_key="suggestions")

In [59]:
writing_style_prompt = PromptTemplate(
    input_variables=["prompt1","document"],
    template="Analyze document based on objectives given as \n\n {prompt1} \n\n Document: \n\n {document} \n\n Analysis:"
)
writing_style_chain = LLMChain(llm=llm, prompt=writing_style_prompt, output_key="writing")

In [60]:
research_quality_prompt = PromptTemplate(
    input_variables=["prompt2","document","writing"],
    template="Analyze document based on objectives given as \n\n {prompt1} \n\n Document: \n\n {document} \n\n also consider previous output\n\n Previous Output:\n\n {writing}"
)
research_quality_chain = LLMChain(llm=llm, prompt=research_quality_prompt, output_key="quality")

In [61]:
flaw_report_prompt = PromptTemplate(
    input_variables=["prompt3","document","quality"],
    template="Analyze document based on objectives given as \n\n {prompt1} \n\n Document: \n\n {document} \n\n also consider previous output\n\n Previous Output:\n\n {quality}"
)
flaw_report_chain = LLMChain(llm=llm, prompt=flaw_report_prompt, output_key="flaws")

In [62]:
suggestions_prompt = PromptTemplate(
    input_variables=["prompt4","document","flaws"],
    template="Analyze document based on objectives given as \n\n {prompt1} \n\n Document: \n\n {document} \n\n also consider previous output\n\n Previous Output:\n\n {flaws}"
)
suggestions_chain = LLMChain(llm=llm, prompt=suggestions_prompt, output_key="suggestions")

In [63]:
seq_chain = SequentialChain(
    chains=[writing_style_chain, research_quality_chain, flaw_report_chain, suggestions_chain],
    input_variables=["document","prompt1","prompt2","prompt3","prompt4"],
    output_variables=["writing","quality","flaws","suggestions"]
)

In [64]:
input_data = {
    "document" : content_chunks[0],
    "prompt1" : prompt1_txt,
    "prompt2" : prompt2_txt,
    "prompt3" : prompt3_txt,
    "prompt4" : prompt4_txt,
}

In [65]:
result = seq_chain.invoke(input_data)

In [66]:
result.keys()

dict_keys(['document', 'prompt1', 'prompt2', 'prompt3', 'prompt4', 'writing', 'quality', 'flaws', 'suggestions'])

In [67]:
print(result.get('flaws'))

 and colloquialisms could be removed for a more professional tone.

**Recommendations:**
- Ensure consistent definition and explanation of specialized terms and acronyms throughout the proposal.
- Review and revise transitions between sections to improve overall coherence and flow.
- Remove any informal phrases and colloquialisms for a more formal tone.

\section{Assessment of Writing Style}

\subsection{Clarity: 8/10}
- Technical concepts and methodologies are explained clearly and concisely.
- Language is accessible to an interdisciplinary audience while maintaining technical rigor.
- Acronyms and specialized terms are consistently defined and explained, but there are a few inconsistencies.

\subsection{Precision: 9/10}
- Writing avoids vague or ambiguous statements.
- Objectives, methodologies, and outcomes are described with sufficient detail and specificity.
- Claims are supported with appropriate data, evidence, or references.

\subsection{Coherence and Flow: 7/10}
- Document fol

In [68]:
print(result.get('suggestions'))

}
- Writing is formal and consistent with academic and scientific standards.
- Tone reflects the ambition and significance of the research without overstating claims.
- Some informal phrases and colloquialisms could be removed for a more professional tone.

\subsection{Engagement and Persuasiveness: 9/10}
- Proposal captures the reader's attention with compelling arguments.
- Societal, scientific, and economic impacts of the research are effectively communicated.
- Unique aspects of the research are emphasized to distinguish it from similar initiatives.

\subsection{Grammar, Syntax, and Formatting: 8/10}
- Sentences are well-constructed, grammatically correct, and free of typographical errors.
- Formatting enhances readability, with consistent headings, bullet points, and lists.
- Visuals, tables, and charts are integrated effectively into the narrative, but there may be room for improvement in their placement and use.

\subsection{Summary of Ratings and Findings}

\begin{tabular}{ |p{

In [70]:
input_data_writing = {
    "document" : content_chunks[0],
    "prompt1" : prompt1_txt}
print(writing_style_chain.invoke(input_data_writing))

{'document': 'Chapter2. ProbabilisticKnowledgeRepresentation\nP(X|Y)=P(Y|X)P(X)\nP(Y)(2.6)\nBayes’ theorem allows to convert an abductive reasoning task into a deductive one\nand vice versa. It plays a fundamental role in Bayesian networks, which model\nuncertain causal relationships among random variables. Applying Bayes’ theorem\nallows to turn a causal model into a diagnostic reasoning task.\nInference It\nPosterior\nInferenceis obviously straightforward to convert a propositional logic KB into a\nprobabilistic KB by introducing a Boolean random variable for every atomic proposi-\ntion inΣ. The joint probability distribution P(Σ)then can be stored in a contingency\ntable holding the co-occurrences of the respective atomic events. Making use of\nthe chain rule, the law of total probability and Bayes’ theorem, one can computethe posterior probability\nP(Q|E)of any arbitrary query Q⊆Σgiven any arbitrary\nevidence E⊆Σ. Such a posterior belief can be computed by the canonical inference\n

In [96]:
from langchain.chains.base import Chain
from typing import Any, Dict, List
from pydantic import BaseModel, Field
from langchain_openai.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

# Initialize the LLM
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    openai_api_key=OPENAI_API_KEY
)

class CustomSequentialChain(Chain, BaseModel):
    llm: OpenAI = Field(...)
    steps: List[Dict[str, Any]] = Field(...)
    output_dir: str = Field(...)

    class Config:
        arbitrary_types_allowed = True

    @property
    def input_keys(self) -> List[str]:
        """
        Input keys required for the first step and any static inputs.
        """
        all_input_keys = set()
        for step in self.steps:
            all_input_keys.update(step["input_keys"])
        return list(all_input_keys)

    @property
    def output_keys(self) -> List[str]:
        """
        Output keys of all steps.
        """
        return [step["output_key"] for step in self.steps]

    def _call(self, inputs: Dict[str, Any]) -> Dict[str, Any]:
        """
        Executes each step sequentially.
        """
        step_inputs = inputs.copy()
        step_outputs = {}

        for i, step in enumerate(self.steps):
            prompt = step["prompt"]
            input_keys = step["input_keys"]
            output_key = step["output_key"]

            # Collect inputs for this step
            formatted_inputs = {key: step_inputs[key] for key in input_keys}
            formatted_prompt = prompt.format(**formatted_inputs)

            # Convert the formatted prompt into a HumanMessage
            messages = [HumanMessage(content=formatted_prompt)]

            # Run the ChatOpenAI instance with messages
            result = self.llm(messages).content
            step_outputs[output_key] = result

            # Add this step's output to the inputs for the next step
            step_inputs[output_key] = result

        return step_outputs

In [97]:
steps = [
    {
        'prompt' : writing_style_prompt,
        "input_keys" : ["document","prompt1"],
        "output_key" : "writing",
    },
    {
        'prompt' : research_quality_prompt,
        "input_keys" : ["document","prompt2"],
        "output_key" : "quality",
    },
    {
        'prompt' : flaw_report_prompt,
        "input_keys" : ["document","prompt3"],
        "output_key" : "flaws",
    },
    {
        'prompt' : suggestions_prompt,
        "input_keys" : ["document","prompt4"],
        "output_key" : "suggestions",
    }
]
output_dir = "output_files"
input_data = {
    "document" : content_chunks[0],
    "prompt1" : prompt1_txt,
    "prompt2" : prompt2_txt,
    "prompt3" : prompt3_txt,
    "prompt4" : prompt4_txt,
}

In [98]:
custom_chain = CustomSequentialChain(llm=llm, steps=steps, output_dir=output_dir)

AttributeError: 'ChatOpenAI' object has no attribute 'get'

In [91]:
outputs = custom_chain(input_data)

NotFoundError: Error code: 404 - {'error': {'message': 'This is a chat model and not supported in the v1/completions endpoint. Did you mean to use v1/chat/completions?', 'type': 'invalid_request_error', 'param': 'model', 'code': None}}